<a href="https://colab.research.google.com/github/saskinosie/CalibrateAI-5-12-25/blob/main/%5Bcalibrateai%5D_hack_day_micro_conference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semantic Search with Weaviate's Query Agent, LlamaIndex and Comet

##Pre-requisites
For this workshop you will need:

* A (free) Weaviate Cloud (WCD) account
* A cluster set up in WCD
* The REST endpoint for your cluster
* Your cluster Admin API key
* An OpenAI API key

In this workshop we will create a Retrieval Augmented Generation system leveraging Weaviate's Query Agent, LlamaIndex and Comet.


We’re utilizing LlamaIndex to transform full-text PDF research articles into manageable, structured text chunks. These chunks are enhanced with metadata and section detection logic, then uploaded into a Weaviate vector database to support semantic search over our collection of space medicine literature using natural languge with Weaviate's query agent. In the subsequent woerkshop, we will use Comet's end-to-end model evaluation platform to benchmark our RAG system.

### Link to slide deck
https://docs.google.com/presentation/d/1UbDpA0dhuHiiSu5vsiv_PPA9HzgzZKcRn7t4zuoFzzA/edit?usp=sharing

### Link to repo and instructions for copllecting REST endpoint and API keys
https://github.com/saskinosie/CalibrateAI-5-12-25

### Link to G-Drive folder with research articles
 https://drive.google.com/drive/folders/18iu8lGJ0SEZcISkUqc20pecGrb61Mo7s?usp=drive_link

In [37]:
!pip install llama-index pymupdf weaviate-client weaviate-agents


In [38]:
import fitz  # PyMuPDF
import json
import re
import requests
from llama_index.core import Document
from llama_index.core.node_parser import HierarchicalNodeParser
import weaviate

In [39]:
from google.colab import userdata

WEAVIATE_URL = userdata.get("WEAVIATE_URL")
WEAVIATE_API_KEY = userdata.get("WEAVIATE_API_KEY")
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

print("Weaviate URL:", WEAVIATE_URL)
print("Weaviate API Key:", WEAVIATE_API_KEY)


Weaviate URL: https://rwxzavyuspepzg2fkhjag.c0.us-west3.gcp.weaviate.cloud
Weaviate API Key: IwgDlvGkKqzCylsFoLh6wAuPgYqY9bvyp7yR


In [40]:
client = weaviate.connect_to_weaviate_cloud(
    WEAVIATE_URL,
    auth_credentials=weaviate.AuthApiKey(WEAVIATE_API_KEY),
    headers={
        "X-OpenAI-Api-Key": OPENAI_API_KEY
    }
)

In [41]:
assert client.is_ready(), "Weaviate client is not ready. Check credentials and endpoint."


In [42]:
client.is_ready()

True

In [43]:
from weaviate.classes.config import Configure

if client.collections.exists("SpaceMedResearch"):
    client.collections.delete("SpaceMedResearch")

client.collections.create(
    name = "SpaceMedResearch",
    vectorizer_config= [
            Configure.NamedVectors.text2vec_weaviate(
                name="main_vector",
                model="Snowflake/snowflake-arctic-embed-l-v2.0",
                source_properties=["title", "content"],
            )
        ],
    )


In [48]:
gdrive_links = [
    "https://drive.google.com/file/d/1bNX5nZTif8roMK1bFaJmHF6wxapi5YDg/view?usp=sharing",
    "https://drive.google.com/file/d/1FZkvMOyTP_-kSIyx9VewaV9tP_XwpZXK/view?usp=drive_link",
    "https://drive.google.com/file/d/1jfcCLHmAazvs7DnAhd3jS0LOb7qOctMc/view?usp=drive_link",
    "https://drive.google.com/file/d/1K8D6VOe2aAX6tIfJWzF2-9zWbqH0C_wp/view?usp=drive_link",
    "https://drive.google.com/file/d/12ee59tcUcxotC1NFfz0EaLNWAAqakKDk/view?usp=drive_link",
    "https://drive.google.com/file/d/115LBMKIobYRdqKWqL2uR1VWoPS5zqoZq/view?usp=drive_link",
    "https://drive.google.com/file/d/1CcjaMYUIQNJ2S4nFGHIh0hpkeG158-Ag/view?usp=drive_link",
    "https://drive.google.com/file/d/1eR6rTQcYw_q4Lob2JFB2_cnND9U2VivA/view?usp=drive_link",
    "https://drive.google.com/file/d/1Gw9UQGNIcDTLpCaamYm4WoeYqUVlKsAG/view?usp=drive_link",
    "https://drive.google.com/file/d/1E961JtImN2eis_IxK5EZS3JaqhXkyhK8/view?usp=drive_link",
    "https://drive.google.com/file/d/1G5xQ10Ijjhrnm_Uq_hl2OgXNaKERfB3G/view?usp=drive_link",
    "https://drive.google.com/file/d/1u-nmLQIvBdcRomCo__yvoCCVNHV3AiKg/view?usp=drive_link",
    "https://drive.google.com/file/d/1cfF_cRkvfaTw5BTpiMxalw0Bc-IdBBnO/view?usp=drive_link",
    "https://drive.google.com/file/d/1WSMqabWY4pElGrQjVVkkaJ8NNEuP6T10/view?usp=drive_link",
    "https://drive.google.com/file/d/11bCbFObW-51XE0lMS3sz5-VYvWsxctRm/view?usp=drive_link",
    "https://drive.google.com/file/d/13AFfg8doORRytR3IXfTOmL1vpp6hsCuG/view?usp=drive_link",
    "https://drive.google.com/file/d/1k8QYuAsyzkMTJA-KzPrOX2BToKmzSr3l/view?usp=drive_link",
    "https://drive.google.com/file/d/1HQJ-WPOXE_CT4bVmuZ4LaZ5GKK8MIgXB/view?usp=sharing",
    "https://drive.google.com/file/d/1sqeJmEmP-3eTTFgTbxTSmS69kiH-K46H/view?usp=sharing"


]

In [49]:
# PDF wrangling function and chunk setup

def download_google_drive_pdf(share_url, output_folder="downloads"):
    os.makedirs(output_folder, exist_ok=True)
    file_id_match = re.search(r"/d/([^/]+)", share_url)
    if not file_id_match:
        raise ValueError(f"Invalid Google Drive URL: {share_url}")
    file_id = file_id_match.group(1)
    download_url = f"https://drive.google.com/uc?export=download&id={file_id}"
    response = requests.get(download_url)
    pdf_path = os.path.join(output_folder, f"{file_id}.pdf")
    with open(pdf_path, "wb") as f:
        f.write(response.content)
    return pdf_path

def extract_text(filepath):
    doc = fitz.open(filepath)
    return "\n".join(page.get_text("text") for page in doc)

def extract_title(text):
    candidate_block = text[:1000]
    lines = [line.strip() for line in candidate_block.split("\n") if line.strip()]
    for i, line in enumerate(lines):
        if line.lower() != line and len(line.split()) > 5 and not line.endswith(":") and i < 5:
            return line
    return "Unknown Title"

def slugify(text):
    text = text.lower()
    text = re.sub(r"[^\w\s-]", "", text)
    text = re.sub(r"\s+", "-", text)
    return text.strip("-")

def detect_section(text_chunk, chunk_index=0):
    lowered = text_chunk.lower()
    if "introduction" in lowered[:150] or chunk_index == 0:
        return "Introduction"
    elif "methods" in lowered[:150] or "materials and methods" in lowered[:150]:
        return "Methods"
    elif "results" in lowered[:150]:
        return "Results"
    elif "discussion" in lowered[:150]:
        return "Discussion"
    elif "conclusion" in lowered[:150]:
        return "Conclusion"
    else:
        return "Unknown"

def chunk_for_weaviate(text, title=None):
    from llama_index.core.node_parser import HierarchicalNodeParser
    from llama_index.core.text_splitter import SentenceSplitter

    if not title:
        title = extract_title(text)
    slug = slugify(title)
    document = Document(text=text, metadata={"title": title, "slug": slug})

    # Create hierarchical parser directly without parameters
    # The latest version may not accept parameters in from_defaults()
    parser = HierarchicalNodeParser.from_defaults()

    # Configure it after creation if needed
    # This approach is more compatible with different versions

    nodes = parser.get_nodes_from_documents([document])

    # Further chunk the nodes if they're too large using SentenceSplitter
    text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=50)
    smaller_nodes = []
    for node in nodes:
        split_texts = text_splitter.split_text(node.text)
        for i, split_text in enumerate(split_texts):
          smaller_nodes.append({
              "text": split_text,
              "metadata": {
                  "title": title,
                  "slug": slug,
                  "section": detect_section(split_text, chunk_index=i)
                  }
              })


    return smaller_nodes



In [50]:
# Chunking Function

def process_gdrive_links(gdrive_links, output_path="demo_chunks.json"):
    all_chunks = []
    for link in gdrive_links:
        print(f"Processing: {link}")
        try:
            pdf_path = download_google_drive_pdf(link)
            text = extract_text(pdf_path)
            chunks = chunk_for_weaviate(text)
            all_chunks.extend(chunks)
        except Exception as e:
            print(f"❌ Failed to process {link}: {e}")

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(all_chunks, f, ensure_ascii=False, indent=2)
    print(f"\n✅ Saved {len(all_chunks)} chunks to {output_path}")

In [51]:
# Chunk PDFs
import os
process_gdrive_links(gdrive_links, output_path="demo_chunks.json")

Processing: https://drive.google.com/file/d/1bNX5nZTif8roMK1bFaJmHF6wxapi5YDg/view?usp=sharing
Processing: https://drive.google.com/file/d/1FZkvMOyTP_-kSIyx9VewaV9tP_XwpZXK/view?usp=drive_link
Processing: https://drive.google.com/file/d/1jfcCLHmAazvs7DnAhd3jS0LOb7qOctMc/view?usp=drive_link
Processing: https://drive.google.com/file/d/1K8D6VOe2aAX6tIfJWzF2-9zWbqH0C_wp/view?usp=drive_link
Processing: https://drive.google.com/file/d/12ee59tcUcxotC1NFfz0EaLNWAAqakKDk/view?usp=drive_link
Processing: https://drive.google.com/file/d/115LBMKIobYRdqKWqL2uR1VWoPS5zqoZq/view?usp=drive_link
Processing: https://drive.google.com/file/d/1CcjaMYUIQNJ2S4nFGHIh0hpkeG158-Ag/view?usp=drive_link
Processing: https://drive.google.com/file/d/1eR6rTQcYw_q4Lob2JFB2_cnND9U2VivA/view?usp=drive_link
Processing: https://drive.google.com/file/d/1Gw9UQGNIcDTLpCaamYm4WoeYqUVlKsAG/view?usp=drive_link
Processing: https://drive.google.com/file/d/1E961JtImN2eis_IxK5EZS3JaqhXkyhK8/view?usp=drive_link
Processing: https://dri

In [52]:
# Function to batch upload to Weaviate (including UUIDs)
def bulk_upload_space_chunks_to_weaviate(json_file_path, collection_name="SpaceMedResearch"):
    import weaviate
    from google.colab import userdata
    from weaviate.util import generate_uuid5

    # Client initialization
    WEAVIATE_URL = userdata.get("WEAVIATE_URL")
    WEAVIATE_API_KEY = userdata.get("WEAVIATE_API_KEY")
    OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

    client = weaviate.connect_to_weaviate_cloud(
        WEAVIATE_URL,
        auth_credentials=weaviate.AuthApiKey(WEAVIATE_API_KEY),
        headers={
            "X-OpenAI-Api-Key": OPENAI_API_KEY
        }
    )

    docs_collection = client.collections.get(collection_name)

    with open(json_file_path, "r", encoding="utf-8") as f:
        chunks = json.load(f)

    successful_uploads = 0

    with docs_collection.batch.fixed_size(batch_size=100, concurrent_requests=2) as batch:
        for i, chunk in enumerate(chunks):
            text = chunk.get("text", "")
            metadata = chunk.get("metadata", {})

            # Create a unique ID by combining title with chunk index and first 20 chars of text
            unique_id = f"{metadata.get('title', 'unknown')}-chunk-{i}-{text[:20]}"
            uid = generate_uuid5(unique_id)

            batch.add_object(
                properties={
                    "content": text,
                    "title": metadata.get("title", "unknown"),
                    "slug": metadata.get("slug", "unknown"),
                    "section": metadata.get("section", "unknown")
                },
                uuid=uid
            )
            successful_uploads += 1

            # Progress indicator
            if i % 500 == 0 and i > 0:
                print(f"Progress: {i}/{len(chunks)} chunks processed")

            if batch.number_errors > 10:
                print("❌ Too many errors during batch import — stopping early.")
                break

    # Verify the actual count in the collection
    collection_count = docs_collection.aggregate.over_all().total_count
    print(f"✅ Uploaded {successful_uploads} chunks to Weaviate from {json_file_path}")
    print(f"✅ Collection now contains {collection_count} objects")

In [53]:
# Batch upload
bulk_upload_space_chunks_to_weaviate("/content/demo_chunks.json")

Progress: 500/5089 chunks processed
Progress: 1000/5089 chunks processed
Progress: 1500/5089 chunks processed
Progress: 2000/5089 chunks processed
Progress: 2500/5089 chunks processed
Progress: 3000/5089 chunks processed
Progress: 3500/5089 chunks processed
Progress: 4000/5089 chunks processed
Progress: 4500/5089 chunks processed
Progress: 5000/5089 chunks processed
✅ Uploaded 5089 chunks to Weaviate from /content/demo_chunks.json
✅ Collection now contains 5089 objects


In [54]:
from weaviate.classes.init import Auth
# Try importing from weaviate-agents
from weaviate_agents.query import QueryAgent

# Instantiate agent object, and specify the collections to query
qa = QueryAgent(
    client=client, collections=["SpaceMedResearch"]
)

In [55]:
# Perform a query
response = qa.run(
    "What are the greatest health concerns facing astronauts during their time in space and upon their return to earth?"
)
# Print the response
response.display()

╭─────────────────────────────────────────────── 🔍 Original Query ───────────────────────────────────────────────╮
│                                                                                                                 │
│ What are the greatest health concerns facing astronauts during their time in space and upon their return to     │
│ earth?                                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Final Answer ────────────────────────────────────────────────╮
│                                                                                                                 │
│ During their time in space, astronauts face several significant health concerns primarily related to the unique │
│ environment of space. Key issues include genetic, immune system, and metabolic function alterations due to      │
│ exposure to space radiation, restricted diet, reduced physical activity, disrupted circadian rhythms, and       │
│ weightlessness. These factors can lead to mitochondrial dysfunction, immunological stress, vascular changes and │
│ fluid shifts, and cognitive performance decline. Molecular changes such as alterations in telomere length, gene │
│ regulation, and genome integrity have also been observed, which may increase long-term health risks.            │
│ Additionally, space missions cause fluid to shift toward the head, which affects vascular pressure and          │
│ contributes to ocular problems.                                                                                 │
│                                                                                                                 │
│ Upon return to Earth, astronauts face one of the greatest physiological challenges: re-adapting to gravity and  │
│ Earth conditions. This period is marked by exaggerated or impaired cardiovascular and musculoskeletal responses │
│ as well as heightened stress and inflammatory reactions. Astronauts commonly experience spaceflight-associated  │
│ neuro-ocular syndrome (SANS), which includes symptoms like optic disc edema, hyperopic shifts, retina-related   │
│ issues, and other ocular symptoms linked to the headward fluid shifts sustained during spaceflight. Telomere    │
│ length tends to shorten after flight, linking to risks akin to aging-related diseases such as cardiovascular    │
│ disease and cancer. The recovery period also involves concerns about long-term consequences for cardiovascular, │
│ ocular, and cognitive health, with ongoing research highlighting the need for personalized health monitoring    │
│ and potential countermeasures to mitigate these risks for longer and more distant future missions.              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 1/2 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['health concerns facing astronauts during space missions'],                                        │
│     filters=[[]],                                                                                               │
│     filter_operators='AND',                                                                                     │
│     collection='SpaceMedResearch'                                                                               │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 2/2 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['health issues astronauts face upon return to Earth'],                                             │
│     filters=[[]],                                                                                               │
│     filter_operators='AND',                                                                                     │
│     collection='SpaceMedResearch'                                                                               │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 📊 No Aggregations Run                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='8e224630-fbeb-57fc-93de-74155cf4275a' collection='SpaceMedResearch'                               │
│  - object_id='b5e577ab-4f4c-537b-99eb-afe6d03b79e3' collection='SpaceMedResearch'                               │
│  - object_id='5b8ce84d-663a-55fe-a6e2-eb9d725921f5' collection='SpaceMedResearch'                               │
│  - object_id='e59d1d3c-9bfe-57a9-81ab-914f35d9c302' collection='SpaceMedResearch'                               │
│  - object_id='b6e41a5d-2538-5b49-a0c7-075c6c4e69b2' collection='SpaceMedResearch'                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

   📊 Usage Statistics   
┌────────────────┬──────┐
│ LLM Requests:  │ 4    │
│ Input Tokens:  │ 5816 │
│ Output Tokens: │ 480  │
│ Total Tokens:  │ 6296 │
└────────────────┴──────┘

Total Time Taken: 8.45s

In [56]:
# Perform a query
response = qa.run(
    "What is tyeh best cookie recipe in the world?"
)
# Print the response
response.display()

╭─────────────────────────────────────────────── 🔍 Original Query ───────────────────────────────────────────────╮
│                                                                                                                 │
│ What is tyeh best cookie recipe in the world?                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Final Answer ────────────────────────────────────────────────╮
│                                                                                                                 │
│ There are no relevant search or aggregation results detailing any cookie recipes, let alone the best cookie     │
│ recipe in the world. Therefore, based on the provided information, it is not possible to determine or recommend │
│ the best cookie recipe.                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 1/1 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['best cookie recipe'],                                                                             │
│     filters=[[]],                                                                                               │
│     filter_operators='AND',                                                                                     │
│     collection='SpaceMedResearch'                                                                               │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 📊 No Aggregations Run                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── ⚠️ Missing Information: ─────────────────────────────────────────────╮
│                                                                                                                 │
│ - Any cookie recipe details                                                                                     │
│ - Recommendations or rankings of cookie recipes                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

   📊 Usage Statistics   
┌────────────────┬──────┐
│ LLM Requests:  │ 3    │
│ Input Tokens:  │ 3086 │
│ Output Tokens: │ 168  │
│ Total Tokens:  │ 3254 │
└────────────────┴──────┘

Total Time Taken: 4.48s

In [57]:
# Perform a query
response = qa.run(
    "What stress is common in pilots?"
)
# Print the response
response.display()

╭─────────────────────────────────────────────── 🔍 Original Query ───────────────────────────────────────────────╮
│                                                                                                                 │
│ What stress is common in pilots?                                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Final Answer ────────────────────────────────────────────────╮
│                                                                                                                 │
│ Common stress factors among pilots include work-related pressures that can lead to mental health symptoms and   │
│ potentially psychiatric illness. Specific stressors for pilots encompass repeated proficiency simulator checks, │
│ intermittent medical exams, total flight hours obligations, employer pressure, and the significant              │
│ responsibility associated with flying. Additional challenges faced by aircrew include environmental factors     │
│ such as hypobaria (low atmospheric pressure), pressurization issues, sedentary conditions, jet lag, and dietary │
│ concerns related to high-caloric foods. These combined elements contribute to the broad spectrum of stress      │
│ experienced by pilots in their professional duties.                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 1/1 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['common stress factors in pilots'],                                                                │
│     filters=[[]],                                                                                               │
│     filter_operators='AND',                                                                                     │
│     collection='SpaceMedResearch'                                                                               │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 📊 No Aggregations Run                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='5eebc114-c7e8-5abc-ac2f-7c3912711e18' collection='SpaceMedResearch'                               │
│  - object_id='487fb746-fef3-50ac-8adf-286d7af1d302' collection='SpaceMedResearch'                               │
│  - object_id='3ccb8170-c56f-583f-bc74-07915c6fa558' collection='SpaceMedResearch'                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

   📊 Usage Statistics   
┌────────────────┬──────┐
│ LLM Requests:  │ 3    │
│ Input Tokens:  │ 3058 │
│ Output Tokens: │ 231  │
│ Total Tokens:  │ 3289 │
└────────────────┴──────┘

Total Time Taken: 4.75s

In [58]:
# Perform a query for articles with contradictory information
response = qa.run(
    "Do all astronauts who engage in long-duration spaceflight develop structural changes in the eyes (Spaceflight-Associated Neuro-ocular Syndrome, or SANS)?"
)
# Print the response
response.display()

╭─────────────────────────────────────────────── 🔍 Original Query ───────────────────────────────────────────────╮
│                                                                                                                 │
│ Do all astronauts who engage in long-duration spaceflight develop structural changes in the eyes                │
│ (Spaceflight-Associated Neuro-ocular Syndrome, or SANS)?                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Final Answer ────────────────────────────────────────────────╮
│                                                                                                                 │
│ Not all astronauts who engage in long-duration spaceflight develop structural changes in the eyes associated    │
│ with Spaceflight-Associated Neuro-ocular Syndrome (SANS). Evidence indicates that about 40% of astronauts have  │
│ experienced one or more ocular changes after long-duration missions on the International Space Station (ISS).   │
│ These structural changes include optic disc edema, choroidal folds, globe flattening, hyperopic shifts,         │
│ cotton-wool spots, and a slight decrease in axial eye length. The severity and incidence of these changes vary  │
│ among individuals. The exact cause and mechanisms behind SANS remain unclear, which has hindered the            │
│ development of effective countermeasures. Therefore, while SANS is a significant health risk for some           │
│ astronauts, it does not affect all individuals undertaking long-duration spaceflight.                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 1/2 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['astronauts long-duration spaceflight Spaceflight-Associated Neuro-ocular Syndrome development'],  │
│     filters=[[]],                                                                                               │
│     filter_operators='AND',                                                                                     │
│     collection='SpaceMedResearch'                                                                               │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 2/2 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=[                                                                                                   │
│         'structural changes in the eyes associated with Spaceflight-Associated Neuro-ocular Syndrome',          │
│         'eye changes seen in Spaceflight-Associated Neuro-ocular Syndrome',                                     │
│         'ocular morphology alterations in Spaceflight-Associated Neuro-ocular Syndrome'                         │
│     ],                                                                                                          │
│     filters=[[], [], []],                                                                                       │
│     filter_operators='OR',                                                                                      │
│     collection='SpaceMedResearch'                                                                               │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 📊 Aggregations Run 1/2 ────────────────────────────────────────────╮
│                                                                                                                 │
│ AggregationResultWithCollection(                                                                                │
│     search_query='Spaceflight-Associated Neuro-ocular Syndrome (SANS) long-duration spaceflight',               │
│     groupby_property=None,                                                                                      │
│     aggregations=[                                                                                              │
│         TextPropertyAggregation(                                                                                │
│             property_name='content',                                                                            │
│             metrics=<TextMetrics.COUNT: 'COUNT'>,                                                               │
│             top_occurrences_limit=None                                                                          │
│         )                                                                                                       │
│     ],                                                                                                          │
│     filters=[                                                                                                   │
│         TextPropertyFilter(                                                                                     │
│             property_name='content',                                                                            │
│             operator=<ComparisonOperator.LIKE: 'LIKE'>,                                                         │
│             value='*Spaceflight-Associated Neuro-ocular Syndrome*'                                              │
│         )                                                                                                       │
│     ],                                                                                                          │
│     collection='SpaceMedResearch'                                                                               │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 📊 Aggregations Run 2/2 ────────────────────────────────────────────╮
│                                                                                                                 │
│ AggregationResultWithCollection(                                                                                │
│     search_query='structural eye changes after long-duration spaceflight',                                      │
│     groupby_property=None,                                                                                      │
│     aggregations=[                                                                                              │
│         TextPropertyAggregation(                                                                                │
│             property_name='content',                                                                            │
│             metrics=<TextMetrics.COUNT: 'COUNT'>,                                                               │
│             top_occurrences_limit=None                                                                          │
│         ),                                                                                                      │
│         TextPropertyAggregation(                                                                                │
│             property_name='content',                                                                            │
│             metrics=<TextMetrics.TOP_OCCURRENCES: 'TOP_OCCURRENCES'>,                                           │
│             top_occurrences_limit=5                                                                             │
│         )                                                                                                       │
│     ],                                                                                                          │
│     filters=[                                                                                                   │
│         TextPropertyFilter(                                                                                     │
│             property_name='content',                                                                            │
│             operator=<ComparisonOperator.LIKE: 'LIKE'>,                                                         │
│             value='*structural eye change*'                                                                     │
│         ),                                                                                                      │
│         TextPropertyFilter(                                                                                     │
│             property_name='content',                                                                            │
│             operator=<ComparisonOperator.LIKE: 'LIKE'>,                                                         │
│             value='*long-duration spaceflight*'                                                                 │
│         )                                                                                                       │
│     ],                                                                                                          │
│     collection='SpaceMedResearch'                                                                               │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── ⚠️ Missing Information: ─────────────────────────────────────────────╮
│                                                                                                                 │
│ - Exact prevalence data beyond the reported ~40% of astronauts affected                                         │
│ - Detailed mechanisms causing SANS                                                                              │
│ - Long-term outcomes of SANS in astronauts                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='92b1e99f-2e10-50f5-a7cf-96e01e18882d' collection='SpaceMedResearch'                               │
│  - object_id='7fa959eb-5f6a-5b6b-90cc-990c0b13c235' collection='SpaceMedResearch'                               │
│  - object_id='ed315c46-2ed6-5a88-b60e-bf3cb553e123' collection='SpaceMedResearch'                               │
│  - object_id='7a58ed26-c684-5489-ada9-5d67f4c8aafc' collection='SpaceMedResearch'                               │
│  - object_id='2d103556-0fb5-5eaf-882c-528610fdf3d2' collection='SpaceMedResearch'                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

   📊 Usage Statistics    
┌────────────────┬───────┐
│ LLM Requests:  │ 6     │
│ Input Tokens:  │ 14663 │
│ Output Tokens: │ 695   │
│ Total Tokens:  │ 15358 │
└────────────────┴───────┘

Total Time Taken: 10.27s

In [59]:
# Perform a query for articles with contradictory information
response = qa.run(
    "What are the proposed mechanisms behind SANS, and do researchers agree on whether fluid shifts and intracranial pressure are the primary causes?"
)
# Print the response
response.display()

╭─────────────────────────────────────────────── 🔍 Original Query ───────────────────────────────────────────────╮
│                                                                                                                 │
│ What are the proposed mechanisms behind SANS, and do researchers agree on whether fluid shifts and intracranial │
│ pressure are the primary causes?                                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Final Answer ────────────────────────────────────────────────╮
│                                                                                                                 │
│ Spaceflight Associated Neuro-ocular Syndrome (SANS) is a condition experienced by astronauts characterized by   │
│ ocular and neuro-ocular changes such as optic disc edema, globe flattening, choroidal folds, and vision         │
│ changes. The proposed mechanisms behind SANS primarily involve cephalad fluid shifts caused by microgravity,    │
│ which may lead to alterations in intracranial pressure (ICP). These fluid shifts are thought to cause           │
│ structural and functional changes in neuro-ocular tissues and possibly brain functional organization. However,  │
│ despite these hypotheses, the exact origin and underlying cause of SANS remain unclear.                         │
│                                                                                                                 │
│ Research indicates that SANS not only affects ocular structures but also impacts brain functional connectivity, │
│ particularly in visual and multisensory integration networks, suggesting a broader neurophysiological           │
│ alteration. Some studies have explored resting-state brain activity and functional connectivity differences in  │
│ individuals who develop SANS, pointing to possible predisposing factors beyond fluid shifts and ICP.            │
│                                                                                                                 │
│ Regarding consensus, there is no full agreement among researchers that fluid shifts and increased intracranial  │
│ pressure are the sole or primary causes of SANS. While these factors are central hypotheses, the exact          │
│ mechanisms are not fully understood, and the condition’s pathophysiology involves complex interactions that are │
│ still under investigation. Further research is needed to clarify these mechanisms, including the role of        │
│ genetic, biochemical, and cerebral functional factors.                                                          │
│                                                                                                                 │
│ In summary, fluid shifts and intracranial pressure changes are proposed and influential mechanisms behind SANS, │
│ but researchers have not reached consensus that these are the definitive primary causes due to the complexity   │
│ and incomplete understanding of the syndrome.                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 1/2 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['proposed mechanisms behind SANS'],                                                                │
│     filters=[[]],                                                                                               │
│     filter_operators='AND',                                                                                     │
│     collection='SpaceMedResearch'                                                                               │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 2/2 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=[                                                                                                   │
│         'researchers agreement on fluid shifts as causes of Spaceflight Associated Neuro-ocular Syndrome',      │
│         'researchers agreement on intracranial pressure as causes of Spaceflight Associated Neuro-ocular        │
│ Syndrome'                                                                                                       │
│     ],                                                                                                          │
│     filters=[[], []],                                                                                           │
│     filter_operators='AND',                                                                                     │
│     collection='SpaceMedResearch'                                                                               │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 📊 No Aggregations Run                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── ⚠️ Missing Information: ─────────────────────────────────────────────╮
│                                                                                                                 │
│ - Specific detailed mechanisms beyond fluid shifts and ICP                                                      │
│ - Consensus statements or reviews explicitly addressing researcher agreement                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='90079f47-00d4-5f18-b61e-d25e2f8ffb39' collection='SpaceMedResearch'                               │
│  - object_id='0a6ffb58-aabf-5332-9dc1-f97729c66bc9' collection='SpaceMedResearch'                               │
│  - object_id='92b1e99f-2e10-50f5-a7cf-96e01e18882d' collection='SpaceMedResearch'                               │
│  - object_id='ed315c46-2ed6-5a88-b60e-bf3cb553e123' collection='SpaceMedResearch'                               │
│  - object_id='fe500fa7-397e-5aed-bbeb-bf64808ba903' collection='SpaceMedResearch'                               │
│  - object_id='7a58ed26-c684-5489-ada9-5d67f4c8aafc' collection='SpaceMedResearch'                               │
│  - object_id='baddfb84-e9c8-58da-bf89-88a11df28013' collection='SpaceMedResearch'                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

   📊 Usage Statistics   
┌────────────────┬──────┐
│ LLM Requests:  │ 4    │
│ Input Tokens:  │ 6587 │
│ Output Tokens: │ 553  │
│ Total Tokens:  │ 7140 │
└────────────────┴──────┘

Total Time Taken: 9.58s